In [9]:
from pandas_profiling import ProfileReport
import numpy as np
import pandas as pd
from sklearn import linear_model
import sklearn.metrics as metrics
import statsmodels.api as sm
import matplotlib.pyplot as plt

import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2OGradientBoostingEstimator

import cx_Oracle as ora
import pyodbc

h2o.init()

training_columns = ['acquisition_week',	'acquisition_month',	'new_deactivation',	'deactivation_reason_fraud',	'deactivation_reason_stolen',	'deactivation_reason_other',	'deactivation_week',	'deactivation_month',	'deactivated',	'active',	'status',	'lifetime_in_days',	'new_contract',	'rate_plan_name',	'rate_plan_monthly_fee',	'subscription_type',	'subscription_prepaid',	'subscription_postpaid',	'subscription_hybrid',	'device_type',	'device_smartphone',	'device_feature_phone',	'device_basic_phone',	'device_tablet',	'account_balance',	'new_bad_debt',	'bad_debt',	'voice_outgoing_duration_in_minutes',	'voice_outgoing_number_of_calls',	'voice_outgoing_billed_revenue',	'voice_outgoing_unique_contacts',	'voice_outgoing_international_duration_in_minutes',	'voice_outgoing_international_number_of_calls',	'voice_outgoing_international_billed_revenue',	'voice_outgoing_international_unique_contacts',	'voice_incoming_duration_in_minutes',	'voice_incoming_number_of_calls',	'voice_incoming_unique_contacts',	'voice_incoming_international_duration_in_minutes',	'voice_incoming_international_number_of_calls',	'voice_incoming_international_unique_contacts',	'sms_outgoing_number_of_messages',	'sms_outgoing_billed_revenue',	'sms_outgoing_unique_contacts',	'unique_contacts',	'data_mb_used',	'data_mb_uploaded',	'data_mb_downloaded',	'data_active',	'data_roaming_mb_used',	'data_roaming_mb_downloaded',	'products_used',	'products_revenue',	'products_voice_packages_bought',	'products_voice_packages_revenue',	'products_data_packages_bought',	'products_data_packages_revenue',	'products_packages_revenue',	'products_packages_active',	'revenue',	'recharges_amount',	'recharges_count',	'network_active',	'top_cell',	'top_city',	'top_region',	'top_network_technology',	'gross_date',	'contract_start_dt',	'contract_end_dt',	'contract_length',	'payment_expiration_date',	'days_of_late_payment',	'bad_debt_amount_more_0_days',	'bad_debt_amount_more_5_days',	'bad_debt_amount_more_10_days',	'bad_debt_amount_more_11_days',	'total_data_quota_in_mb',	'total_data_balance_left_in_mb',	'total_data_used_in_mb',	'total_data_used_percent',	'has_total_data_balance',	'plan_data_quota_in_mb',	'plan_data_balance_left_in_mb',	'plan_data_used_in_mb',	'plan_data_used_percent',	'has_plan_data_balance',	'invoice_amount',	'other_payments_amount',	'other_payments_quantity',	'postpaid_unique_contacts',	'postpaid_voice_outgoing_unique_contacts',	'postpaid_voice_incoming_unique_contacts',	'postpaid_sms_outgoing_unique_contacts',	'postpaid_sms_incoming_unique_contacts', 'arpu_in_2_months',	'edad',	'potential_bancarizado',	'tigo_usa',	'tigoshop_user',	'tipo_pago',	'usa_asterisco_dar',	'dispositovo_nombre_categoria',	'has_postpaid',	'has_home', 'actual_bank_user', 'payment_type_tigo_money', 'payment_type_payment_gateway','cashin','cashin_amount','cashout','cashout_amount','peer2peer','peer2peer_amount','service_payment','service_payment_amount','recharge','recharge_amount','monto_transaccion','wallet_30','wallet_60','wallet_90'] #,	'cliente_identidad'
# eliminated features: 'device_name',	'device_vendor_name',	'device_os_name',

# response_column = 'DIGITAL_LTR'
response_column = 'flag'

user = 'BI'
passw = 'dwhprodbi20$'
server = 'srvdwhdb.celtel.net:1521/dwh.celtel.net'

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 41 mins
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,9 months and 14 days !!!
H2O_cluster_name:,H2O_from_python_emil_ordonez_b7yfc2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,10.54 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [ ]:
##########READING FROM DATABASE - Mobile Metrics
cxn = pyodbc.connect("DSN=hivedwh",autocommit=True)

dataList = []

with open('query2.sql', 'r') as file:
    query = file.read().replace('\n', ' ')

i = 1
    
for chunk in pd.read_sql_query(sql=query,con=cxn, chunksize = 200000):
    dataList.append(chunk)
    i = i+1

data = pd.concat(dataList, ignore_index=True)
print('dataframe len:'.format(len(data)))

dataList = []

data.msisdn = data.msisdn.astype(int)

#del data

In [ ]:
##########READING FROM DATABASE - MFS Metrics
cxn_ora = ora.connect(user, passw, server)

dataList = []

with open('10_ora-mfs_trx.sql', 'r') as file:
    query = file.read().replace('\n', ' ')
    
for chunk in pd.read_sql(sql=query,con=cxn_ora, chunksize = 200000):
    dataList.append(chunk)

data_mfstrx01 = pd.concat(dataList, ignore_index=True)
data_mfstrx01.columns= data_mfstrx01.columns.str.lower()
print('dataframe len:'.format(len(data_mfstrx01)))

cxn_ora.close()

cxn_ora = ora.connect(user, passw, server)

dataList = []

with open('11_ora-wallet_users.sql', 'r') as file:
    query = file.read().replace('\n', ' ')

print(query)
    
for chunk in pd.read_sql(sql=query,con=cxn_ora, chunksize = 200000):
    dataList.append(chunk)

data_mfswalletusers = pd.concat(dataList, ignore_index=True)
data_mfswalletusers.columns= data_mfswalletusers.columns.str.lower()
print('dataframe len:'.format(len(data_mfswalletusers)))

dataList = []

cxn_ora.close()
#del data

In [24]:
# data = pd.read_csv("data4.csv",sep=',')#, encoding='utf-16')
# data.msisdn = data.msisdn.astype(int)
# data = data[data['status'] == 'ACTIVE']

cc_data = pd.read_csv("211115-contracargos.csv",sep='\t')#,nrows=100000)
data_join = data.merge(cc_data, on='msisdn', how='left', indicator=False)
del cc_data

#data_mfswalletusers = pd.read_csv("mfs_wallet_users210812.csv",sep=',')#,nrows=2000000)
data_join = data_join.merge(data_mfswalletusers, on='sub_billing_id', how='left', indicator=False)
#del data_mfswalletusers

#data_mfstrx01 = pd.read_csv("mfs_trx210812.csv",sep=',')#,nrows=2000000)
data_join = data_join.merge(data_mfstrx01, on='sub_billing_id', how='left', indicator=False)
#del data_mfstrx01

##FILL NA Values
values = {'flag': 0, 'monto': 0,'wallet_30': 0,'wallet_60': 0,'wallet_90': 0,'cashin': 0,'cashin_amount': 0,'cashout': 0,'cashout_amount': 0,'peer2peer': 0,'peer2peer_amount': 0,'service_payment': 0,'service_payment_amount': 0,'recharge': 0,'recharge_amount': 0,'tiene_movimientos_mfs': 0,'monto_transaccion': 0,'gross_date':'2999-12-31',	'contract_start_dt':'2999-12-31',	'contract_end_dt':'2999-12-31',	'contract_length':0,	'payment_expiration_date':'2999-12-31',	'days_of_late_payment':0,	'bad_debt_amount_more_0_days':0,	'bad_debt_amount_more_5_days':0,	'bad_debt_amount_more_10_days':0,	'bad_debt_amount_more_11_days':0,	'total_data_quota_in_mb':0,	'total_data_balance_left_in_mb':0,	'total_data_used_in_mb':0,	'total_data_used_percent':0,	'has_total_data_balance':0,	'plan_data_quota_in_mb':0,	'plan_data_balance_left_in_mb':0,	'plan_data_used_in_mb':0,	'plan_data_used_percent':0,	'has_plan_data_balance':0,	'postpaid_unique_contacts':0,	'postpaid_voice_outgoing_unique_contacts':0,	'postpaid_voice_incoming_unique_contacts':0,	'postpaid_sms_outgoing_unique_contacts':0,	'postpaid_sms_incoming_unique_contacts':0, 'arpu_in_2_months': 0,	'edad': 99,	'potential_bancarizado': 0,	'tigo_usa': 0,	'tigoshop_user': 0,	'usa_asterisco_dar': 0,	'has_postpaid': 0,	'has_home': 0,'actual_bank_user':0, 'payment_type_tigo_money':0, 'payment_type_payment_gateway':0}
data_join.fillna(value=values,inplace=True)

# data_join.to_csv('fraud_data_joined211115.csv',index=False)

# postpaid_data = data_join[data_join['tipo_pago'].isin(['PSTPAID','HBRDPAID'])]
# postpaid_data.to_csv('fraud_data_joined211115_psp.csv',index=False)
# del postpaid_data

# prepaid_data = data_join[data_join['tipo_pago'] == 'PRPAID']
# prepaid_data.to_csv('fraud_data_joined211115_prp.csv',index=False)
# del prepaid_data

del data

In [43]:
##### WALLET DATA FRAME PREPARATION


#data_wallet = pd.read_csv("fraud_data_joined3.csv",sep=',')#, encoding='utf-16')
#data.msisdn = data_wallet.msisdn.astype(int)
data_wallet = data_join[data_join['wallet_90'] == 1]
data_wallet = data_wallet[data_wallet['status'] == 'ACTIVE']

#data_wallet.to_csv('fraud_data_joined210812_wallet90.csv',index=False)

In [36]:
# h2o.init()

#### TRAINING AND PRINT TESTING:

datah2o_wallet = h2o.H2OFrame(data_wallet)
# datah2o_wallet = h2o.import_file('fraud_data_joined3_wallet90.csv')
print("Import finalizado")

datah2o_wallet.fillna()
print("Fill NA finalizado")

datah2o_wallet[['flag']]=datah2o_wallet[['flag']].asfactor()
datah2o_wallet[['cliente_identidad']]=datah2o_wallet[['cliente_identidad']].asfactor()
datah2o_wallet[['potential_bancarizado']]=datah2o_wallet[['potential_bancarizado']].asfactor()
datah2o_wallet[['actual_bank_user']]=datah2o_wallet[['actual_bank_user']].asfactor()
print("ASFACTOR finalizado")

train_wallet, test_wallet = datah2o_wallet.split_frame(ratios=[0.7])
print("Ratios finalizado")

##Random Forest
model_wallet = H2ORandomForestEstimator()
print("Training Started")
# model = H2OGradientBoostingEstimator(nfolds=3)
# Train model
model_wallet.train(x=training_columns, y=response_column, training_frame=train_wallet)

model_wallet.save_model_details

##For Tableau Analysis:
model_wallet.predict(test_wallet).as_data_frame().to_csv('prp_test_predict_wallet211115.csv')#head()
test_wallet[['sub_billing_id','msisdn','monto','flag', 'potential_bancarizado', 'actual_bank_user']].as_data_frame().to_csv('prp_test_real_wallet211115.csv')
model_wallet._model_json['output']['variable_importances'].as_data_frame().to_csv("prp_variables_importances_wallet211115.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Import finalizado
Fill NA finalizado
ASFACTOR finalizado
Ratios finalizado
Training Started


c:\users\emil.ordonez\.conda\envs\ccfraudtest01\lib\site-packages\h2o\estimators\estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [bad_debt_amount_more_10_days, plan_data_used_in_mb, plan_data_used_percent, total_data_balance_left_in_mb, bad_debt_amount_more_5_days, deactivation_reason_stolen, deactivated, wallet_90, deactivation_reason_fraud, new_deactivation, bad_debt_amount_more_11_days, has_total_data_balance, new_contract, plan_data_quota_in_mb, has_plan_data_balance, total_data_used_in_mb, active, bad_debt_amount_more_0_days, total_data_quota_in_mb, plan_data_balance_left_in_mb, total_data_used_percent, deactivation_month, deactivation_reason_other, deactivation_week, status]
  warnings.warn(mesg["message"], RuntimeWarning)


drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [44]:
#### Wallet Users Predict: (Production)

model_wallet.predict(datah2o_wallet).as_data_frame().to_csv('prp_complete_predict_wallet211115.csv')#head()
datah2o_wallet[['sub_billing_id','msisdn','monto','flag', 'potential_bancarizado', 'actual_bank_user']].as_data_frame().to_csv('prp_complete_real_wallet211115.csv')

drf prediction progress: |████████████████████████████████████████████████| 100%


In [42]:
#Full Base predict:

#del datah2o_prp
#del test_prp
#del train_prp

####Correr una de las 2 cargas de dataframe (desde archivo o desde otro dataframe)
print("Complete Base Reading Started")
#datah2o_complete = h2o.import_file('fraud_data_joined210812.csv')
#datah2o_complete = h2o.H2OFrame(data_join)
print("Import finalizado")

print("Complete Base Output Started")
#datah2o_complete[['sub_billing_id','msisdn','monto','flag', 'potential_bancarizado', 'actual_bank_user']].as_data_frame().to_csv('prp_complete_real211115.csv')
print("Export Real finalizado")

print("Complete Base Predict Started")
model_wallet.predict(datah2o_complete).as_data_frame().to_csv('prp_test_predict_complete211115.csv')#head()
print("Predict y Export finalizado")
#test_prp[['sub_billing_id','msisdn_x','monto','flag', 'potential_bancarizado', 'actual_bank_user']].as_data_frame().to_csv('prp_test_real4.csv')
#model_prp._model_json['output']['variable_importances'].as_data_frame().to_csv("prp_variables_importances4.csv")

Complete Base Reading Started
Import finalizado
Complete Base Output Started
Export Real finalizado
Complete Base Predict Started
drf prediction progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_81d34b887b7fc9d82872691da7074876 failed with an exception: java.lang.IllegalArgumentException: Test/Validation dataset has categorical column 'wallet_30' which is real-valued in the training data
stacktrace: 
java.lang.IllegalArgumentException: Test/Validation dataset has categorical column 'wallet_30' which is real-valued in the training data
	at hex.Model.adaptTestForTrain(Model.java:1469)
	at hex.Model.adaptTestForTrain(Model.java:1316)
	at hex.Model.adaptTestForTrain(Model.java:1312)
	at hex.Model.score(Model.java:1606)
	at water.api.ModelMetricsHandler$1.compute2(ModelMetricsHandler.java:403)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1575)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:974)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1477)
	at jsr166y.ForkJoinWorkerThread.run(

In [ ]:
#h2o.shutdown()

In [ ]:
#del datah2o_prp
#del model_prp
#del test_prp
#del train_prp

In [ ]:
whos